In [28]:
import pandas as pd
import numpy as np
import pickle as p

In [29]:
data_all = pd.read_csv('data_all.csv')

# Test with Subset of Data
np.random.seed(41)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=100, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()

# # Test with All Data
# data_subset = data_all.copy()

In [30]:
item_subset

[31,
 2259,
 1203,
 5073,
 102,
 5501,
 3818,
 4270,
 3767,
 197,
 3821,
 7108,
 1502,
 2501,
 2589,
 4273,
 4272,
 1002,
 4728,
 2,
 5469,
 1204,
 7113,
 5068,
 3771,
 4411,
 3773,
 16017,
 3772,
 11,
 7008,
 5581,
 2731,
 350,
 3737,
 1232,
 21,
 7158,
 12,
 203,
 200,
 196,
 2551,
 2258,
 5820,
 1503,
 202,
 7115,
 1152,
 3727,
 4978,
 3766,
 3817,
 3726,
 3731,
 7041,
 5771,
 3717,
 2315,
 5666,
 16016,
 3721,
 16001,
 7966,
 2541,
 7129,
 4268,
 3723,
 7079,
 3046,
 1003,
 1225,
 16019,
 2256,
 3823,
 7080,
 1195,
 3743,
 7128,
 2581,
 2721,
 5069,
 3733,
 7965,
 5074,
 3768,
 1221,
 281,
 4976,
 7009,
 3047,
 16000,
 7575,
 4975,
 204,
 2701,
 1615,
 2887,
 5662,
 284]

### `/train`

In [31]:
import requests
import json

url = 'http://localhost:5000/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': True,
           'project_id': 2
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [32]:
r.json()['result']

'Success'

In [33]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

,avg_sales,item_id,mae_score,mpe_score,r2_score,rmse_score
0,618.707153,2.0,104.738354,0.169286,0.746347,162.553185
1,218.526474,11.0,24.713785,0.113093,0.876857,33.914161
2,303.203735,12.0,36.834535,0.121484,0.870070,49.991249
3,169.172928,21.0,21.221341,0.125442,0.891807,30.361540
4,45.567699,31.0,10.240617,0.224734,0.809420,14.335036
...,...,...,...,...,...,...
95,40.986607,16000.0,10.068232,0.245647,0.621022,13.362869
96,15.621078,16001.0,5.307674,0.339776,0.616283,7.259695
97,52.138657,16016.0,10.749791,0.206177,0.739345,14.616714
98,17.366009,16017.0,3.945526,0.227198,0.663164,5.162955


### `/optimize`

In [ ]:
url = 'http://localhost:5000/optimize/'

payload = {'project_id': 1,
           'constraints': [],
           'population': 100,
           'max_epoch': 200
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

In [ ]:
best_prices = r.json()['result']
best_prices

In [ ]:
import requests
import json

url = 'http://localhost:5000/predict/'
payload = {'prices': best_prices,
           'project_id': 1
           }
headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }
payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)

# What If Prediction
qty = r.json()['qty_estimates']

In [ ]:
qty

In [ ]:
# Calculate Revenue
revenue = 0
for item in best_prices.keys():
    revenue += (best_prices[item]*qty['Qty_'+item])
    
print(revenue)

### `/predict/`

In [25]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'31': 1.85, '102': 9.65, '1203': 3.2, '2259': 13.0, '5073': 3.7}

In [27]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'http://localhost:5000/predict/'

payload = {'prices': a_input, 'project_id': 1}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_102,160.0,70.0
Qty_1203,191.0,138.0
Qty_2259,80.0,38.0
Qty_31,40.0,44.0
Qty_5073,91.0,26.0
